# Rating predictions

user.ratedList
user.rating <-- number of changes will tell us how many contests they've done

- get users who's done at least 11 contests
- \# contests above ELO in the previous x amount of time
- take submission dataframe, take last month above ELO --> get number of questions solved


- how much momentum
 

In [2]:
%matplotlib inline
%load_ext rpy2.ipython

import requests
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir

## Fn: get user submissions

In [3]:
def getUserSubmissions(handle):
    maxcount = '100000000'
    url = 'http://codeforces.com/api/user.status?handle=' + handle + '&from=1&count=' + maxcount

    r = requests.get(url).json()['result']

    arr = []
    keys = ['testset', 'passedTestCount', 'author', 'relativeTimeSeconds', 
            'language', 'memoryBytes', 'timeMilliseconds', 'problem_name', 'problem_index',
            'problem_tags', 'points', 'contestID', 'verdict', 'id', 'participantType', 'startTimeSeconds']

    for rr in r:
        temp = dict.fromkeys(keys)
        temp['author'] = rr['author']['members'][0]['handle']
        temp['startTimeSeconds'] = rr['creationTimeSeconds']
        if 'startTimeSeconds' not in rr['author']:
            temp['participantType'] = 'GYM'
        else:
            temp['participantType'] = rr['author']['participantType']
        temp['id'] = rr['id']
        temp['verdict'] = rr['verdict']
        temp['contestID'] = rr['contestId']
        if 'points' not in rr['problem']:
            temp['points'] = 0
        else:
            temp['points'] = rr['problem']['points']
        temp['problem_tags'] = rr['problem']['tags']
        temp['problem_index'] = rr['problem']['index']
        temp['problem_name'] = rr['problem']['name']
        temp['timeMilliseconds'] = rr['timeConsumedMillis']
        temp['memoryBytes'] = rr['memoryConsumedBytes']
        temp['language'] = rr['programmingLanguage']
        temp['relativeTimeSeconds'] = rr['relativeTimeSeconds']
        temp['passedTestCount'] = rr['passedTestCount']
        temp['testset'] = rr['testset']

        arr.append(temp)
    return pd.DataFrame.from_dict(arr)

## Fn: get user rating history

In [4]:
def getUserRatingHistory(handle):
    url = 'http://codeforces.com/api/user.rating?handle=' + handle

    r = requests.get(url).json()['result']
    return pd.DataFrame.from_dict(r)

## Fn: get list of users

In [5]:
url = 'http://codeforces.com/api/user.ratedList?activeOnly=true'
users = requests.get(url).json()['result']

allHandles = []
for u in users:
    allHandles.append(u['handle'])

# API Calls to get user info 

In [11]:
ratings_dfs = []

last_idx = 7965

for idx, user in enumerate(allHandles[last_idx:]):
    df = getUserRatingHistory(user)
    if len(df) > 10:
        ratings_dfs.append(df)
        print idx, user, 'kept'
    else:
        print idx, user, 'skipped'
    
    print "ratings_dfs len:", len(ratings_dfs)
    if len(ratings_dfs) >= 10:
        out_df = pd.concat(ratings_dfs)
        print "writing", len(out_df), "lines to file..."
        with open('rating_histories.csv', 'a') as f:
            out_df.to_csv(f, index=False, header=False, encoding='utf-8')
        ratings_dfs = []

# save anything left in out_df that's not a multiple of 10
out_df = pd.concat(ratings_dfs)
with open('rating_histories.csv', 'a') as f:
    out_df.to_csv(f, index=False, header=False, encoding='utf-8')

0 asad_IT kept
ratings_dfs len: 1
1 mr.baraa kept
ratings_dfs len: 2
2 Hedgehogy kept
ratings_dfs len: 3
